In [312]:
import cPickle as pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import pandas as pd
from pylab import *
from sklearn import preprocessing
import collections
from sklearn.grid_search import GridSearchCV

In [4]:
kegg_queries = pickle.load( open( "../data/kegg_queries.p", "rb" ) )

In [53]:
keggs = kegg_queries.keys()
#kegg_lists = [str(k) for k in  kegg_queries.values()]
kegg_lists = [str(kegg_queries[kegg]) for kegg in  keggs]

In [206]:
count_vect = CountVectorizer(min_df = 5, analyzer = "word", stop_words="english")
X_train_counts = count_vect.fit_transform(kegg_lists)

In [207]:
X_train_counts.shape

(10193, 15881)

In [208]:

tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)

In [209]:
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(10193, 15881)

In [88]:
df = pd.read_csv("../data/keggs_RF_conf_scores.csv")
df.head()

kegg     level_1                  level_2 BRITE_id  \
0  K00001  Metabolism                 Overview      NaN   
1  K00002  Metabolism                 Overview      NaN   
2  K00003  Metabolism                 Overview      NaN   
3  K00004  Metabolism  Carbohydrate metabolism      NaN   
4  K00005  Metabolism  Carbohydrate metabolism      NaN   

                                                   X     LS001     LS002  \
0                      K00001(alcohol_dehydrogenase)  0.005732  0.006140   
1              K00002(alcohol_dehydrogenase_(NADP+))  0.000382  0.000409   
2                   K00003(homoserine_dehydrogenase)  0.000855  0.001034   
3  K00004((R,R)-butanediol_dehydrogenase_/_diacet...  0.000091  0.000106   
4                     K00005(glycerol_dehydrogenase)  0.000167  0.000236   

      LS003     LS004     LS005           ...             HE.SRS017103  \
0  0.006243  0.005962  0.008011           ...                 0.003359   
1  0.000610  0.000533  0.000313           ...                 0.000071   
2  0.001125  0.001074  0.001224           ...                 0.000249   
3  0.000149  0.000116  0.000064           ...                 0.000008   
4  0.000223  0.000203  0.000218           ...                 0.000030   

   HE.SRS017433  HE.SRS018427  HE.SRS018656  HE.SRS020328  HE.SRS020869  \
0      0.004281      0.003052      0.003625      0.002673      0.003999   
1      0.000014      0.000248      0.000649      0.000151      0.000157   
2      0.000131      0.000351      0.000676      0.000430      0.000318   
3      0.000008      0.000014      0.000039      0.000068      0.000010   
4      0.000004      0.000034      0.000036      0.000055      0.000065   

   HE.SRS022713  HE.SRS016585.suspect   over            kegg.selected  
0      0.003734          8.220359e-03  0.542               0 Test set  
1      0.000000          4.812945e-07  0.218               0 Test set  
2      0.000050          1.209999e-06  0.294  1 Used for KS selection  
3      0.000006          4.127393e-04  0.202  1 Used for KS selection  
4      0.000000          2.158647e-04  0.028  1 Used for KS selection  

[5 rows x 70 columns]

In [128]:
kegg_labels = []
for kegg in keggs:
    if kegg in list(df.kegg):
        kegg_score = df[df.kegg== kegg]["over"].item()
        if kegg_score > 0.75: 
            kegg_labels.append("over")
        elif kegg_score < 0.25:
            kegg_labels.append("under")
        else:
            kegg_labels.append("neither")
    else:
        kegg_labels.append("unknown")

In [138]:
len(kegg_labels)

10193

In [239]:
le = preprocessing.LabelEncoder()
y_labels = le.fit_transform(kegg_labels)

In [240]:
counter=collections.Counter(y_labels)
counter

Counter({0: 7507, 2: 2057, 1: 448, 3: 181})

In [246]:
counter=collections.Counter(kegg_labels)
counter

Counter({'neither': 7507, 'under': 2057, 'over': 448, 'unknown': 181})

In [241]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_counts, y_labels)
predicted = clf.predict(X_train_counts)
mean(predicted == y_labels)

0.72706759540861376

In [242]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='log', penalty='l2',alpha=1e-1, n_iter=100, random_state=42)
clf.fit(X_train_counts, y_labels)
predicted = clf.predict(X_train_counts)
mean(predicted == y_labels)

0.84979888158540173

In [243]:
from sklearn import metrics

In [245]:
print(metrics.classification_report(y_labels, predicted))

             precision    recall  f1-score   support

          0       0.86      0.96      0.91      7507
          1       1.00      0.14      0.24       448
          2       0.79      0.59      0.68      2057
          3       1.00      0.99      1.00       181

avg / total       0.85      0.85      0.83     10193



In [248]:
from sklearn import cross_validation
from sklearn.pipeline import Pipeline

In [308]:
text_clf = Pipeline([('vect', CountVectorizer(min_df = 5, analyzer = "word", stop_words="english")),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf', SGDClassifier(loss='log', penalty='l2',
                                           alpha=1e-5, n_iter=100, random_state=42))])

In [313]:
parameters = {'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-1, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7)}

In [309]:
#scores = cross_validation.cross_val_score(text_clf, kegg_lists, y_labels, cv = 10, scoring="f1_weighted", n_jobs=-1)
gs_clf = GridSearchCV(text_clf, parameters, cv=10, n_jobs=1, scoring = "f1_weighted")

In [310]:
mean(scores)

0.76953054964137024

In [311]:
sqrt(var(scores))

0.011550940579644047